In [1]:
import pandas as pd
import numpy as np

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [3]:
df1.drop(columns=['Opening','Opening.1','Production company','.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.'],inplace=True)

In [4]:
df2.drop(columns=['Opening','Opening.1','Production company','Ref.'],inplace=True)
df3.drop(columns=['Opening','Opening.1','Production company','Ref.'],inplace=True)
df4.drop(columns=['Opening','Opening.1','Production company','Ref.'],inplace=True)


In [5]:
df1 = df1.rename({'Title':'movie_title','Cast and crew':'cast_and_crew'},axis=1)
df2 = df2.rename({'Title':'movie_title','Cast and crew':'cast_and_crew'},axis=1)
df3 = df3.rename({'Title':'movie_title','Cast and crew':'cast_and_crew'},axis=1)
df4 = df4.rename({'Title':'movie_title','Cast and crew':'cast_and_crew'},axis=1)

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
final_df = pd.concat([df1, df2,df3,df4], ignore_index=True, sort=False)

In [8]:
final_df.isnull().sum()

movie_title      0
cast_and_crew    0
dtype: int64

In [9]:
final_df.duplicated().sum()

0

In [10]:
s1 = list(final_df['cast_and_crew'][59])
s2 = list(final_df['cast_and_crew'][60])
s3 = list(final_df['cast_and_crew'][154])
s4 = list(final_df['cast_and_crew'][197])
s5 = list(final_df['cast_and_crew'][216])

s1[23] = ';'
s2[22]= ';'
s3[20] = ';'
s4[23]= ')'
s4[24] = ';'
s5[47] = ';'

In [11]:
final_df['cast_and_crew'][59] = "".join(s1)
final_df['cast_and_crew'][60] = "".join(s2)
final_df['cast_and_crew'][154] = "".join(s3)
final_df['cast_and_crew'][197] = "".join(s4)
final_df['cast_and_crew'][216] = "".join(s5)

In [12]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director);")[0]
    elif "(directors)" in x:
        return x.split(" (directors);")[0]
    elif "(director/\u200bscreenplay)" in x:
        return x.split(" (director/\u200bscreenplay);")[0]
    elif " (directors/screenplay);" in x:
        return x.split(" (directors/screenplay);")[0]
    else:
        return x.split(" (director/screenplay);")[0]

In [13]:
final_df['director_name'] = final_df['cast_and_crew'].map(lambda x: get_director(str(x)))

In [14]:
a1 = list(final_df['cast_and_crew'][154])
a1[61]=';'

In [15]:
final_df['cast_and_crew'][154] = "".join(a1)

In [16]:
final_df = final_df.drop(index=[197,239,250]).reset_index()

In [17]:
def get_actor1(x):
    if '(screenplay)' in x:
        return ((x.split("(screenplay); ")[-1]).split(',')[0])
    elif "(co-director/screenplay)" in x:
        return ((x.split("(co-director/screenplay); ")[-1]).split(',')[0])
    elif "(director)" in x:
        return ((x.split("(director); ")[-1]).split(',')[0])
    else:
        return ((x.split("; ")[-1]).split(',')[0])

In [18]:
final_df['actor_1_name'] = final_df['cast_and_crew'].map(lambda x: get_actor1(str(x)))

In [19]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [20]:
final_df['actor_2_name'] = final_df['cast_and_crew'].map(lambda x: get_actor2(str(x)))

In [21]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [22]:
final_df['actor_3_name'] = final_df['cast_and_crew'].map(lambda x: get_actor3(str(x)))

In [23]:
final_df.drop(columns='cast_and_crew',inplace=True)

In [24]:
final_df.drop(columns='index',inplace=True)

In [25]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
import requests
import json

In [26]:
from tmdbv3api import TMDb
tmdb = TMDb()
tmdb.api_key = '4f482c58561643445d7340848e36b5e8'

In [27]:
movie=Movie()
search = movie.search('Stratton')

for res in search:
    print(res.id)
    print(res.title)
    print(res.genre_ids)

348389
Stratton
[28, 53]
43440
The Stratton Story
[18, 10749]
441863
David Stratton: A Cinematic Life
[99]


In [28]:
from tmdbv3api import Movie
movie = Movie()
def get_genre(x):
    genres = []
    result = movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=4f482c58561643445d7340848e36b5e8'.format(movie_id))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        genre_list = str(genres)
        return genre_list
#         return genre_str.join(genre_list)
    else:
        np.NaN

In [29]:
final_df['Genre'] = final_df['movie_title'].map(lambda x: get_genre(str(x)))

In [30]:
def list_to_string(x):
    if x:
        return x.replace(',', '').replace('[','').replace(']','').replace("'",'')
    else:
        return np.NaN

In [31]:
final_df['genres'] = final_df['Genre'].map(lambda x: list_to_string(x))

In [32]:
final_df.drop(columns='Genre',inplace=True)

In [33]:
new_df18 = final_df.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [34]:
new_df18.isnull().sum()

director_name     0
actor_1_name      0
actor_2_name      8
actor_3_name     25
genres            2
movie_title       0
dtype: int64

In [35]:
new_df18['actor_2_name'] = new_df18['actor_2_name'].replace(np.nan, 'unknown')
new_df18['actor_3_name'] = new_df18['actor_3_name'].replace(np.nan, 'unknown')
new_df18['genres'] = new_df18['genres'].replace(np.nan, 'unknown')

In [36]:
director_name = []
for i in new_df18['director_name']:
    director_name.append(i.replace(" ",""))

In [37]:
actor_1_name = []
for i in new_df18['actor_1_name']:
    actor_1_name.append(i.replace(" ",""))

In [38]:
actor_2_name = []
for i in new_df18['actor_2_name']:
    actor_2_name.append(i.replace(" ",""))

In [39]:
actor_3_name = []
for i in new_df18['actor_3_name']:
    actor_3_name.append(i.replace(" ",""))

In [40]:
new_df18['director_name'] = director_name
new_df18['actor_1_name'] = actor_1_name
new_df18['actor_2_name'] = actor_2_name
new_df18['actor_3_name'] = actor_3_name

In [41]:
new_df18['genresres']= new_df18['genres'].str.lower()

In [42]:
new_df18['comb'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' '+ new_df18['actor_3_name'] + ' '+ new_df18['director_name'] +' ' + new_df18['genres']

In [43]:
new_df18

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,AdamRobitel,LinShaye,AngusSampson,LeighWhannell,horror mystery thriller,Insidious: The Last Key,LinShaye AngusSampson LeighWhannell AdamRobite...
1,LaurenWolkstein,AlexPettyfer,JamesFreedson-Jackson,EmilyAlthaus,thriller drama,The Strange Ones,AlexPettyfer JamesFreedson-Jackson EmilyAlthau...
2,SimonWest,DominicCooper,AustinStowell,GemmaChan,action thriller,Stratton,DominicCooper AustinStowell GemmaChan SimonWes...
3,WarwickThornton,BryanBrown,SamNeill,unknown,drama history western,Sweet Country,BryanBrown SamNeill unknown WarwickThornton dr...
4,JaumeCollet-Serra,LiamNeeson,VeraFarmiga,PatrickWilson,action thriller,The Commuter,LiamNeeson VeraFarmiga PatrickWilson JaumeColl...
5,BabakNajafi,TarajiP.Henson,BillyBrown,DannyGlover,thriller action crime,Proud Mary,TarajiP.Henson BillyBrown DannyGlover BabakNaj...
6,BrettDonowho,BruceWillis,ColeHauser,ShawnAshmore,action,Acts of Violence,BruceWillis ColeHauser ShawnAshmore BrettDonow...
7,TrudieStyler,AlexLawther,AbigailBreslin,BetteMidler,comedy drama,Freak Show,AlexLawther AbigailBreslin BetteMidler TrudieS...
8,SamHoffman,JemaineClement,ElliottGould,IngridMichaelson,comedy,Humor Me,JemaineClement ElliottGould IngridMichaelson S...
9,DanielaThomas,AdrianoCarvalho,LuanaNastas,unknown,drama,Vazante,AdrianoCarvalho LuanaNastas unknown DanielaTho...


In [44]:
new_df18.to_csv('2018_data.csv',index=False)